In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, datasets, optimizers
import os

标准的ReesNet


In [2]:
# GPU 的问题！！
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
tf.random.set_seed(2345)

In [4]:
class BasicBlock(layers.Layer):

    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()

        # padding 对于能够整除的四周均匀补全，对于不能整除的，自适应补全
        # 第一层
        self.conv1 = layers.Conv2D(filter_num, (3,3), strides=stride, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')

        # 第二层
        self.conv2 = layers.Conv2D(filter_num, (3,3), strides=1, padding='same')
        self.bn2 = layers.BatchNormalization()
        
        # 保证短接后的 channels 一致
        if stride != 1:
            # 短接层， identity layer
            self.downsample = Sequential()     # TODO ？？？
            self.downsample.add(layers.Conv2D(filter_num, (1, 1), strides=stride))
        else:
            self.downsample = lambda x:x


    def call(self, inputs, training=None):
        # 前向传播
        # [b, h ,w, c]
        out = self.conv1(inputs)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        identity = self.downsample(inputs)

        output = layers.add([out, identity])
        output = self.relu(output)              # output = tf.nn.relu(output)

        return output

In [5]:
# ResNet 是多个 BasicBlock 顿叠而成
class ResNet(keras.Model):

    def __init__(self, layer_dims, num_calsses=100):   #  layer_dims [2,2,2,2] 每一层的basic block
        super(ResNet, self).__init__()
        
        # 设置预处理层
        self.stem = Sequential([layers.Conv2D(64, (3,3), strides=(1, 1)),
                               layers.BatchNormalization(),
                               layers.Activation('relu'),
                               layers.MaxPooling2D(pool_size=(2,2), strides=(1, 1), padding='same')
                               ])

        self.layers1 = self.build_resblock(64, layer_dims[0])
        # h, w 维会变小
        self.layers2 = self.build_resblock(128, layer_dims[1], stride=2)
        self.layers3 = self.build_resblock(256, layer_dims[2], stride=2)
        self.layers4 = self.build_resblock(512, layer_dims[3], stride=2)

        # 全连接层 output : [b, 512, h, w]，自适应输出用于输出
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_calsses)


    def call(self, inputs, training=None):
        # 前向运算过程，预处理
        x = self.stem(inputs)
        # 4 个 resBlock
        x = self.layers1(x)
        x = self.layers2(x)
        x = self.layers3(x)
        x = self.layers4(x)

        # [b, c]
        x = self.avgpool(x)
        # [b, 100]
        x = self.fc(x)
        return x
    
    # 实现 resblock
    def build_resblock(self, filter_num, blocks, stride=1):
        res_blocks = Sequential()
        # 添加第一层， may down sample
        res_blocks.add(BasicBlock(filter_num, stride))
        # 后面的 BasicBlock 不会下采样  ？？？# TODO？？？
        for _ in range(1, blocks):
            res_blocks.add(BasicBlock(filter_num, stride=1))
        return res_blocks

In [6]:
def resnet18():
    return ResNet([2, 2, 2, 2])   #   1 + 4 * 4 + 1

In [7]:
def resnet34():
    return ResNet([3, 4, 6, 3])

In [8]:
(x, y), (x_val, y_val) = datasets.cifar10.load_data()

In [9]:
def preprocess(x, y):
    # [0-255]  - [-1, 1]
    x = 2 * tf.cast(x, dtype=tf.float32) / 255. - 1
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [10]:
# [10k, 1]  [10k]  [10k , 10]
y = tf.squeeze(y)
y_val = tf.squeeze(y_val)
# 在后面进行 one—hot 编码
# y = tf.one_hot(y, depth=10)
# y_val = tf.one_hot(y_val, depth=10)
x.shape, y.shape, x_val.shape, y_val.shape, x.min(), x.max()

((50000, 32, 32, 3),
 TensorShape([50000]),
 (10000, 32, 32, 3),
 TensorShape([10000]),
 0,
 255)

In [11]:
batchsz = 32

In [12]:
train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.map(preprocess).shuffle(10000).batch(batchsz)

test_db = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_db = test_db.map(preprocess).batch(batchsz)

In [13]:
sample = next(iter(train_db))
print(sample[0].shape, sample[1].shape, tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))

(32, 32, 32, 3) (32,) tf.Tensor(-1.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)


In [14]:
def main():
    model = resnet18()
    model.build(input_shape=(None, 32, 32, 3))
    model.summary()
    optimizer = optimizers.Adam(lr=1e-3)
    
    for epoch in range(10):
        for step, (x, y) in enumerate(train_db):
            with tf.GradientTape() as tape:
                # [b, 32, 32, 3]  =>  [b, 100]
                logits = model(x)
                # print(logits.shape)
                # [b] => [b, 100]
                y_onthot = tf.one_hot(y, depth=100)
                # print(y_onthot.shape)
                loss = tf.losses.categorical_crossentropy(y_onthot, logits, from_logits=True)
                loss = tf.reduce_mean(loss)

            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            if step % 300 ==0:
                print(epoch, step, 'loss', float(loss))

        total_num = 0
        total_correct = 0

        for x, y in test_db:
            logits = model(x)
            prob = tf.nn.softmax(logits, axis=1)
            pred = tf.argmax(prob, axis=1)
            pred = tf.cast(pred, dtype=tf.int32)

            correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
            correct = tf.reduce_sum(correct)
            
            total_num += x.shape[0]
            
            total_correct += int(correct)

        acc = total_correct / total_num
        print(epoch, 'acc', acc)

In [15]:
main()

Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 30, 30, 64)        2048      
_________________________________________________________________
sequential_1 (Sequential)    (None, 30, 30, 64)        148736    
_________________________________________________________________
sequential_2 (Sequential)    (None, 15, 15, 128)       526976    
_________________________________________________________________
sequential_4 (Sequential)    (None, 8, 8, 256)         2102528   
_________________________________________________________________
sequential_6 (Sequential)    (None, 4, 4, 512)         8399360   
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  5130

KeyboardInterrupt: 

In [ ]:
model = resnet18()
model.build(input_shape=(None, 32, 32, 3))
optimizer = optimizers.Adam(lr=1e-3)
total_num = 0
total_correct = 0
# x, y 都不是一张图片，而是一个 batch 即 32 张图片
for x, y in test_db:
    # [b, 32, 32, 3]  =>  [b, 100]  现在 b = 32
    logits = model(x)
    prob = tf.nn.softmax(logits, axis=1)
    pred = tf.argmax(prob, axis=1)
    pred = tf.cast(pred, dtype=tf.int32)

    correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
    correct = tf.reduce_sum(correct)
    
    total_num += x.shape[0]
    print(total_num)
    print(x.shape[0])
    total_correct += int(correct)
    break